##### Copyright 2019 The TensorFlow Authors.

# Deep Conditional Convolutional Generative Adversarial Network

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
# To generate GIFs
!pip install -q imageio

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
from tensorflow.keras import Model
import time
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Model

import tensorflow as tf

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import (
        Activation, BatchNormalization, Concatenate, Dense, Dropout, Multiply,
        Embedding, Flatten, Input, Reshape, LeakyReLU, Conv2D, Conv2DTranspose) 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from IPython import display

### Load and prepare the dataset

You will use the MNIST dataset to train the generator and the discriminator. The generator will generate handwritten digits resembling the MNIST data.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
tfds.list_builders()
builder = tfds.builder("fashion_mnist")
builder.download_and_prepare()

In [ ]:
print(builder.info)

In [ ]:
(train_dataset_raw) = builder.as_dataset(split="train", as_supervised=True)

In [ ]:
def format_example(image, label):
  image = tf.cast(image, tf.float32)
  # scale from [0,255] to [-1,1]
  image = (image - 127.5) / 127.5
  image = tf.image.resize(image, (28, 28))
  return image, label

In [ ]:
train_dataset = train_dataset_raw.map(format_example)

In [ ]:
i = 0
for image, label in train_dataset.take(49):
  # define subplot
  plt.subplot(7, 7, 1 + i)
  # turn off axis
  plt.axis('off')
  # plot raw pixel data
  image = (image + 1.0) / 2.0
  plt.imshow(image[:,:,0], cmap='gray')
  i = i + 1
  print(label)
plt.show()

## Create the models

Both the generator and discriminator are defined using the [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model).

### The Generator

The generator uses `tf.keras.layers.Conv2DTranspose` (upsampling) layers to produce an image from a seed (random noise). Start with a `Dense` layer that takes this seed as input, then upsample several times until you reach the desired image size of 28x28x1. Notice the `tf.keras.layers.LeakyReLU` activation for each layer, except the output layer which uses tanh.

In [ ]:

# Generator CNN model
def generator_model(n_class,z_dim=100):

    model = Sequential()

    model.add(Dense(256 * 7 * 7, input_dim=z_dim,))
    model.add(Reshape((7, 7, 256)))

    model.add(Conv2DTranspose(128, 3, 2, padding='same',))
    model.add(LeakyReLU(alpha =0.01))

    model.add(Conv2DTranspose(64, 3, 1, padding='same',))
    model.add(LeakyReLU(alpha =0.01))
    
    model.add(Conv2DTranspose(32, 3, 1, padding='same',))
    model.add(LeakyReLU(alpha =0.01))

    model.add(Conv2DTranspose(1, 3, 2, padding='same',))
    model.add(LeakyReLU(alpha =0.01))

    return model

# generator input 
def make_generator_model(n_class,z_dim=100):
    # latent input
    z = Input(shape=(z_dim, ))
    # label input
    label = Input(shape=(1, ), dtype='int32')
    # convert label to embedding
    label_embedding = Embedding(n_class, z_dim)(label)

    label_embedding = Flatten()(label_embedding)
    # dot product two inputs
    joined_representation = Multiply()([z, label_embedding])

    generator = generator_model(z_dim)

    conditioned_img = generator(joined_representation)

    model =  Model([z, label], conditioned_img)
    # save model blueprint to image
    plot_model(model,'generator.jpg',show_shapes=True,show_dtype=True)

    return model


Use the (as yet untrained) generator to create an image.

In [ ]:
n_classes = 10
generator = make_generator_model(n_classes,100)
tf.keras.utils.plot_model(generator, show_shapes=True, to_file='generator.png')

In [ ]:
#generator.summary()
noise = tf.random.normal([1, 100])
from numpy.random import randint
noisy_label = randint(0, n_classes, 1)
generated_image = generator([noise, noisy_label], training=False)
plt.imshow((generated_image[0, :, :, 0] + 1) / 2, cmap='gray')

### The Discriminator

The discriminator is a CNN-based image classifier.

In [ ]:
# def make_discriminator_model(n_classes):
#   # label input
#   in_label = Input(shape=(1,))
#   # embedding for categorical input
#   li = Embedding(n_classes, 50)(in_label)
#   # scale up to image dimensions with linear activation
#   n_nodes = (28,28,1)[0] * (28,28,1)[1]
#   li = Dense(n_nodes)(li)
#   # reshape to additional channel
#   li = Reshape(((28,28,1)[0], (28,28,1)[1], 1))(li)
#   # image input
#   in_image = Input(shape=(28,28,1))
#   # concat label as a channel
#   merge = Concatenate()([in_image, li])
#   # downsample
#   fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
#   fe = LeakyReLU(alpha=0.2)(fe)
#   # downsample
#   fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
#   fe = LeakyReLU(alpha=0.2)(fe)
#   # flatten feature maps
#   fe = Flatten()(fe)
#   # dropout
#   fe = Dropout(0.4)(fe)
#   # output
#   out_layer = Dense(1, activation='sigmoid')(fe)
#   # define model
#   model = Model([in_image, in_label], out_layer)
#   return model  


In [ ]:
# discriminator CNN model
def discriminator_model(img_shape):

    model = Sequential()

    model.add(Conv2D(64,3,2,input_shape=(img_shape[0], img_shape[1], img_shape[2] + 1),))
    model.add(LeakyReLU(alpha =0.01))

    model.add(Conv2D(128,3,2,input_shape=img_shape,padding='same',))
    model.add(LeakyReLU(alpha =0.001))

    model.add(Conv2D(128,3,2,input_shape=img_shape,padding='same',))
    model.add(LeakyReLU(alpha =0.001))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model


def make_discriminator_model(n_class,img_shape):
    # image input
    img = Input(shape=img_shape)
    # label input
    label = Input(shape=(1, ), dtype='int32')

    label_embedding = Embedding(n_class, np.prod(img_shape),input_length=1)(label)

    label_embedding = Flatten()(label_embedding)

    label_embedding = Reshape(img_shape)(label_embedding)
    # concatenate the image and label
    concatenated = Concatenate(axis=-1)([img, label_embedding])

    discriminator = discriminator_model(img_shape)

    classification = discriminator(concatenated)

    model = Model([img, label], classification)

    plot_model(model,'discriminator.jpg',show_shapes=True,show_dtype=True)

    return model

In [ ]:
n_classes = 10
img_shape =(28,28,1)
discriminator = make_discriminator_model(n_classes, img_shape)
#tf.keras.utils.plot_model(discriminator, show_shapes=True, to_file='discriminator.png')

Use the (as yet untrained) discriminator to classify the generated images as real or fake. The model will be trained to output positive values for real images, and negative values for fake images.

In [ ]:
#discriminator.summary()
from numpy.random import randint
label = randint(0, n_classes, 1)
print(label)
decision = discriminator([generated_image, label])
print (decision)

## Define the loss and optimizers

Define loss functions and optimizers for both models.


In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

### Discriminator loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss
The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, we will compare the discriminators decisions on the generated images to an array of 1s.

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

The discriminator and the generator optimizers are different since we will train two networks separately.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.002, beta_1=0.5)
metricsAcc=tf.keras.metrics.BinaryAccuracy()

### Save checkpoints
This notebook also demonstrates how to save and restore models, which can be helpful in case a long running training task is interrupted.

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop


In [ ]:
noise_dim = 100
num_examples_to_generate = 100

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
from numpy import asarray
seed_images = tf.random.normal([num_examples_to_generate, noise_dim])
seed_labels = asarray([x for _ in range(10) for x in range(10)])
print(seed_labels)

The training loop begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
#@tf.function
def train_step(images, labels):
    noise_image = tf.random.normal([BATCH_SIZE, noise_dim])
    noise_labels = label = randint(0, n_classes, BATCH_SIZE)
    #print("coming into the function")
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

      real_output = discriminator([images, labels], training=True)
      #print("salida con imagenes reales {:.5f}".format(tf.math.reduce_mean(real_output)))
      generated_images = generator([noise_image, noise_labels], training=True)
      fake_output = discriminator([generated_images, noise_labels], training=True)
      #print("salida con imagenes falsas {:.5f}".format(tf.math.reduce_mean(fake_output)))
      
      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
      
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    j = 0
    for image_batch, label_batch in dataset:
      #print(j)
      train_step(image_batch, label_batch)
      #j = j + 1

    print("A NEW EPOCH")
    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator, epoch + 1, seed_images, seed_labels)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator, epochs, seed_images, seed_labels)

**Generate and save images**


In [ ]:
def generate_and_save_images(model, epoch, test_input_images, test_input_labels):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model([test_input_images, test_input_labels], training=False)
  dict_clothes ={0: "T-shirt/top",1: "Trouser",2: "Pullover",
               3: "Dress",4: "Coat",5: "Sandal",6: "Shirt",
               7: "Sneaker",8: "Bag",9: "Ankle boot"}
  fig = plt.figure(figsize=(10,10))
  pediction_length = predictions.shape[0]

  labels = np.tile(np.arange(0,10),int(pediction_length/10)).reshape(-1,1)
    
  for i in range(predictions.shape[0]):
      plt.tight_layout()

      plt.subplot(10, 10, i+1)
      # scale [-1, 1] to [0, 1]
      image = (predictions[i, :, :, 0] + 1.0) / 2.0
      plt.imshow(image, cmap='gray')
      plt.axis('off')
      label_predicted = labels[i][0]
      plt.title(f'{dict_clothes.get(label_predicted)}')
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
# dict_clothes ={0: "T-shirt/top",1: "Trouser",2: "Pullover",
#                3: "Dress",4: "Coat",5: "Sandal",6: "Shirt",
#                7: "Sneaker",8: "Bag",9: "Ankle boot"}

# # function to plot and save sample images
# def plot_sample_images(epoch ,rows=5,columns=4):

#     z = np.random.normal(0, 1, (rows * columns, z_dim))
#     a =np.arange(0,10)
#     b =np.arange(0,10)

#     labels = np.append(a,b).reshape(-1,1)
    
#     gen_imgs = generator.predict([z, labels])

#     gen_imgs = 0.5 * gen_imgs + 0.5
#     print("Epoch : %d "%(epoch+1))
#     fig, axs = plt.subplots(rows,
#                             columns,
#                             figsize =(50, 20),
#                             sharey=True,
#                             sharex=True)

#     cnt = 0
#     for i in range(rows):
#         for j in range(columns):
#             axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
#             axs[i, j].axis('off')
#             axs[i, j].set_title("Type: %s" % dict_clothes.get(labels[cnt][0]))
#             cnt += 1
#     fig.savefig('image%d.jpg'%(epoch))

## Train the model
Call the `train()` method defined above to train the generator and discriminator simultaneously. Note, training GANs can be tricky. It's important that the generator and discriminator do not overpower each other (e.g., that they train at a similar rate).

At the beginning of the training, the generated images look like random noise. As training progresses, the generated digits will look increasingly real. After about 50 epochs, they resemble MNIST digits. This may take about one minute / epoch with the default settings on Colab.

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
EPOCHS = 1000
train_dataset_shuffled = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train(train_dataset_shuffled, EPOCHS)

Restore the latest checkpoint.

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Create a GIF


In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [ ]:
display_image(EPOCHS)

Use `imageio` to create an animated gif using the images saved during training.

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
  display.Image(filename=anim_file)

If you're working in Colab you can download the animation with the code below:

In [ ]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(anim_file)